# Demo for generating complete LUTs from partial input



In [1]:
from cana.drawing.schema_vis import plot_schemata
from cana.drawing.plot_look_up_table import plot_look_up_table
from cana.utils import fill_out_lut

from cana.boolean_network import BooleanNetwork
from cana.boolean_node import BooleanNode

In [2]:
from cana.datasets.bio import THALIANA

thaliana = THALIANA()

thaliana.nodes[0].look_up_table()

,In:,Out:
0,0000000,0
1,0000001,0
2,0000010,0
3,0000011,0
4,0000100,0
...,...,...
123,1111011,1
124,1111100,1
125,1111101,1
126,1111110,1


## Partial LUTs

These are incomplete effective graphs with '-' signifying the wildcard symbol. Visually inspecting an effective graph doesn't allow us to infer its status of completeness or identify any inherent contradictions within the inputs.

The fill_out_lut() function enables us to check for contradictory output data. The contradictory outputs are marked by '!'

The fill_out_lut() function also checks for missing rules in the LUT, allowing us to correct out data infered from sources and explore state spaces yet to be understood.  These outputs are marked by '?'.

In [13]:
# Using the fill_out_lut function found in utils.py

# example look up tables
partial_luts = [
    [("00--", "0"), ("1--1", "1"), ("11--", "1")],
    [("1--", "1"), ("101", "0"), ("011", "0"), ("01-", "1")],
    [("0--0", "0"), ("1--1", "0"), ("0111", "1"), ("0011", "1")],
    [("1---", "0")],
]
generated_lut = fill_out_lut(partial_luts[3], fill_clashes=True)
generated_lut

[('0000', '?'),
 ('0001', '?'),
 ('0010', '?'),
 ('0011', '?'),
 ('0100', '?'),
 ('0101', '?'),
 ('0110', '?'),
 ('0111', '?'),
 ('1000', '0'),
 ('1001', '0'),
 ('1010', '0'),
 ('1011', '0'),
 ('1100', '0'),
 ('1101', '0'),
 ('1110', '0'),
 ('1111', '0')]

## Instantiating a BooleanNode object using partial LUT

Incorporating the fill_out_lut() function into the instantiation of a single boolean node object under the class BooleanNode, this object will now contain '?' for unspecified output values in the LUT entry. 


In [16]:
# example partial look up tables
partial_luts = [
    [("001-", "0"), ("1--1", "1"), ("11--", "1")],
    [("00--", "0"), ("1--1", "1"), ("110-", "1")],
    [("1--", "1"), ("101", "0"), ("011", "0"), ("01-", "1")],  # will have clashes
    [("0--0", "0"), ("1--1", "0"), ("0111", "1"), ("0011", "1")],
    [("1-01", "1"), ("1-1-", "0"), ("0110", "0"), ("01-1", "1")],
    [("1-01", "1"), ("1-1-", "0"), ("0110", "0"), ("01-1", "?")],
    [("-1--", "0")],
    [("-1--", "1")],
    [("-1--", "?")],
]

# using the from_partial_lut function found in BooleanNode class
for partial_lut in partial_luts:
    generated_node = BooleanNode.from_partial_lut(partial_lut, fill_clashes=True)
    # print(generated_node)
    print(generated_node.outputs)
    # plot_look_up_table(generated_node)
    # plot_schemata(generated_node)  # This will throw an error as the presence of '?' makes it impossible to generate a schemata look up table.

['?', '?', '0', '0', '?', '?', '?', '?', '?', '1', '?', '1', '1', '1', '1', '1']
['0', '0', '0', '0', '?', '?', '?', '?', '?', '1', '?', '1', '1', '1', '?', '1']
Clashing output values for entry: 101
Clashing output values for entry: 011
['?', '?', '1', '1', '1', '1', '1', '1']
['0', '?', '0', '1', '0', '?', '0', '1', '?', '0', '?', '0', '?', '0', '?', '0']
['?', '?', '?', '?', '?', '1', '0', '1', '?', '1', '0', '0', '?', '1', '0', '0']
['?', '?', '?', '?', '?', '?', '0', '?', '?', '1', '0', '0', '?', '1', '0', '0']
['?', '?', '?', '?', '0', '0', '0', '0', '?', '?', '?', '?', '0', '0', '0', '0']
['?', '?', '?', '?', '1', '1', '1', '1', '?', '?', '?', '?', '1', '1', '1', '1']
['?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?']


## Generating missing output values randomly

In [5]:
# filling missing output values randomly with 1 or 0 instead of '?'
partial_lut = [("1-01", "1"), ("1-1-", "0"), ("01--", "0")]
generated_node = BooleanNode.from_partial_lut(
    partial_lut, fill_missing_output_randomly=True
)
print(generated_node)
print(generated_node.outputs)

<BNode(id=0, name='x', k=4, inputs=[1,2,3,4], state='0', outputs=[0,1,0,0...1,1,0,0] constant=False)>
['0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0']


## Generating LUT with specified Node Bias

The BooleanNode object calculates the node bias using the .bias() function. 

```python
generated_node.bias()
```


In [6]:
partial_luts = [
    [("001-", "0"), ("1--1", "1"), ("11--", "1")],
    [("00--", "0"), ("1--1", "1"), ("110-", "1")],
    [("1--", "1"), ("101", "0"), ("011", "0"), ("01-", "1")], # will have clashes
    [("0--0", "0"), ("1--1", "0"), ("0111", "1"), ("0011", "1")],
    [("1-01", "1"), ("1-1-", "0"), ("0110", "0"), ("01-1", "1")],
    [("1-01", "1"), ("1-1-", "0"), ("0110", "0"), ("01-1", "?")],
    [("-1--", "0")],
    [("-10-", "1")],
    [("11-", "?")],
]


for lut in partial_luts:
    node = None
    # generated_node_permutations = None
    node = BooleanNode.from_partial_lut(lut, fill_clashes=True)
    # print(node.outputs)

    generated_node_permutations = node.generate_with_required_bias(
        bias=0.5, limit=50, verbose=True
    )
    # print(generated_node_permuations[0].outputs, "\n")

2.80e+01 possible permutation(s) with a bias of 0.5. This is the closest bias less than or equal to the required bias of 0.5.
3.50e+01 possible permutation(s) with a bias of 0.5. This is the closest bias less than or equal to the required bias of 0.5.
Clashing output values for entry: 101
Clashing output values for entry: 011
1.00e+00 possible permutation(s) with a bias of 0.75. This is the closest achievable bias to the required bias of 0.5.
1.00e+00 possible permutation(s) with a bias of 0.5. This is the closest bias less than or equal to the required bias of 0.5.
3.50e+01 possible permutation(s) with a bias of 0.5. This is the closest bias less than or equal to the required bias of 0.5.
8.40e+01 possible permutation(s) with a bias of 0.5. This is the closest bias less than or equal to the required bias of 0.5.
1.00e+00 possible permutation(s) with a bias of 0.5. This is the closest bias less than or equal to the required bias of 0.5.
4.95e+02 possible permutation(s) with a bias of 0

/data/siyer/CANA/cana/boolean_node.py:1075: UserWarning: Required Node Bias is lower than the minimum achievable bias (0.75) of the node. Generating with the minimum achievable bias.
  warnings.warn(


In [7]:
node = next(generated_node_permutations)
node.outputs

['0', '1', '1', '1', '0', '0', '0', '1']

## Generating LUT with specified Effective connectivity

The BooleanNode object calculates effective connectivity using the .effective_connectivity() function.

```python

generated_node.effective_connectivity()

```
        

Generating a Look Up Table for a node with a specified effective connectivity. 

In [8]:
partial_luts = [
    [("001-", "0"), ("1--1", "1"), ("11--", "1")],
    [("00--", "0"), ("1--1", "1"), ("110-", "1")],
    [("1--", "1"), ("101", "0"), ("011", "0"), ("01-", "1")], # will have clashes
    [("0--0", "0"), ("1--1", "0"), ("0111", "1"), ("0011", "1")],
    [("1-01", "1"), ("1-1-", "0"), ("0110", "0"), ("01-1", "1")],
    [("1-01", "1"), ("1-1-", "0"), ("0110", "0"), ("01-1", "?")],
    [("-1--", "0")],
    [("-1--", "1")],
    [("-1--","?")]
]
partial_lut = partial_luts[0]

for partial_lut in partial_luts:
    generated_nodes = BooleanNode.from_partial_lut(partial_lut, fill_clashes=True)
    print(f"Partial LUT: {generated_nodes.outputs}")
    generated_node_permutations = (
        generated_nodes.generate_with_required_effective_connectivity(effective_connectivity=0.5, epsilon = 0.01, verbose=True)
    )
    perm = next(generated_node_permutations)
    print(perm.outputs)
    print(f"Ec = {perm.effective_connectivity()}.")
    print(perm.is_within_tolerance(effective_connectivity=0.5, epsilon=0.01),"\n")


# # Incorporating the above functions into the from_partial_lut() under BooleanNode class
# generated_node = BooleanNode.from_partial_lut(
#     partial_lut, effective_connectivity=0.45
# )

# plot_look_up_table(generated_node)
# plot_schemata(generated_node)

Partial LUT: ['?', '?', '0', '0', '?', '?', '?', '?', '?', '1', '?', '1', '1', '1', '1', '1']
No. of '?' in output = 8.
Returning a generator of nodes with effective connectivity within 0.01 of 0.5.
['0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1', '1', '1', '1', '1']
Ec = 0.5.
True 

Partial LUT: ['0', '0', '0', '0', '?', '?', '?', '?', '?', '1', '?', '1', '1', '1', '?', '1']
No. of '?' in output = 7.
Returning a generator of nodes with effective connectivity within 0.01 of 0.5.
['0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1', '1', '1', '1', '1']
Ec = 0.5.
True 

Clashing output values for entry: 101
Clashing output values for entry: 011
Partial LUT: ['?', '?', '1', '1', '1', '1', '1', '1']
No. of '?' in output = 2.
Returning a generator of nodes with effective connectivity within 0.01 of 0.5.
['0', '0', '1', '1', '1', '1', '1', '1']
Ec = 0.4166666666666667.
False 

Partial LUT: ['0', '?', '0', '1', '0', '?', '0', '1', '?', '0', '?', '0', '?', '0', '?', '0']
No.

/data/siyer/CANA/cana/boolean_node.py:1266: UserWarning: No node within 0.01 of 0.5 (required effective connectivity) found.
Generating a node with the closest effective connectivity of 0.4166666666666667
  warnings.warn(


In [9]:
automata = {'GKL': [['###10#0', 0], ['1#10###', 1]],
 'GP': [['0##1##0', 0],
  ['0##10##', 0],
  ['0#01###', 0],
  ['##10##1', 1],
  ['1##0##1', 1],
  ['###01#1', 1]],
}

In [10]:
generated_node = BooleanNode.from_partial_lut(automata['GP'])
effective_connectivity = 0.4
bias = None
verbose = True
epsilon = 0.01
shuffle = True  


generated_node_permutations = generated_node.generate_with_required_effective_connectivity(
    effective_connectivity=effective_connectivity,
    bias=bias,
    epsilon=epsilon,
    shuffle=shuffle,
    verbose=verbose,
)



No. of '?' in output = 72.
Returning a generator of nodes with effective connectivity within 0.01 of 0.4.


In [11]:
print(next(generated_node_permutations).effective_connectivity())

0.393945272237683
